#网页爬虫、request、BS4学习

#1.request使用

##1.1获取get请求

In [8]:
import requests 

In [10]:
#无参数get请求
req = requests.get('http://httpbin.org/get')
page = req.text #获取页面文本
#带参数get请求
params = {'q':'杨彦星'}
req = requests.get('http://httpbin.org/get', params = params)
req.url #得到请求的url

u'http://httpbin.org/get?q=%E6%9D%A8%E5%BD%A6%E6%98%9F'

##1.2获取post请求

In [15]:
#含参post请求
params = {'a':'杨','b':'hello'}
req = requests.post('http://httpbin.org/gpost',data = params)
page = req.text #获取页面信息
#提交json格式请求
params = {'a':'杨','b':'hello'}
import json
req = requests.post('http://httpbin.org/post', data=json.dumps(params))

##1.3提交文件

In [17]:
#提交文件 需要files参数
url = 'http://httpbin.org/post'
files = {'file': open('1.jpg', 'rb')}
req = requests.post(url, files=files)

##1.4使用header提交文件

In [25]:
import json
url = 'https://www.baidu.com'
params = {'some': 'data'}
headers = {'content-type': 'application/json'}
req = requests.post(url, data=json.dumps(params), headers=headers)

##1.5响应操作

In [29]:
print req.headers
print req.headers['Content-Type']
print req.status_code

{'content-length': '6120', 'content-encoding': 'gzip', 'set-cookie': 'BAIDUID=2E25687BD52023CEC43C01AD40FBE2E9:FG=1; expires=Sat, 17-Sep-16 10:16:13 GMT; max-age=31536000; path=/; domain=.baidu.com; version=1, __bsi=11822572061416239299_00_0_I_R_6_0303_C02F_N_I_I_0; expires=Fri, 18-Sep-15 10:16:18 GMT; domain=www.baidu.com; path=/', 'accept-ranges': 'bytes', 'expires': 'Sat, 19 Sep 2015 10:16:13 GMT', 'vary': 'Accept-Encoding,User-Agent', 'server': 'bfe/1.0.8.5', 'last-modified': 'Thu, 12 Mar 2015 07:47:33 GMT', 'connection': 'keep-alive', 'etag': '"7e1b-5111298ba8f40"', 'cache-control': 'max-age=86400', 'date': 'Fri, 18 Sep 2015 10:16:13 GMT', 'p3p': 'CP=" OTI DSP COR IVA OUR IND COM "', 'content-type': 'text/html'}
text/html
200


#2.BeautifulSoup

In [34]:
#测试文档
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

In [36]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')
print(soup.prettify())

<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    Elsie
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>


In [74]:
#只能获取到第一个标签
soup.title
#获取标签的属性
print(soup.title.name)
#获取父节点
print(soup.title.parent.name)
#获取属性
soup.p['class']

title
meta


[u'switch_radios']

In [39]:
#获取所有标签
for link in soup.find_all('a'):
    #获取标签下的属性
    print(link.get('href'))

http://example.com/elsie
http://example.com/lacie
http://example.com/tillie


#3.request和BeautifulSoup连接

In [40]:
req = requests.get("http://www.baidu.com")
page = req.text #获取界面文本
soup = BeautifulSoup(page,"html.parser")

#4.网页爬取

In [ ]:
#同上

#5.图片爬取

In [53]:
import os
def saveImage( imgUrl,imgName ="default.jpg" ):
    response = requests.get(imgUrl, stream=True)
    image = response.content
    DstDir=os.getcwd() #保存在当前工作目录下
    print("保存文件"+imgName+"\n")
    try:
        with open(os.path.join(os.getcwd(),'image',imgName) ,"wb") as jpg:
            jpg.write( image)     
            return
    except IOError:
        print("IO Error\n")
        return
    finally:
        jpg.close        

In [55]:
#生成时间戳保证下载的内容文件名不重复
import time
def getT():
    t = time.time()
    s = str(int(t*100%10000000))
    return s

#6.爬取百度贴吧图片

In [71]:
#首先需要请求一个贴吧的主页
main_url = "http://tieba.baidu.com/f"
page_num = 2 #用与计算需要爬取的页数
title_list= [] #保存所有要爬取帖子的链接
for i in range(page_num): 
    params = {"kw" : "python" , "pn":i*50} #通过页码和关键字决定
    req = requests.get(main_url,params=params) #得到一个页面
    page = req.text #得到页面文本
    soup = BeautifulSoup(page,'html.parser')
    a_list = soup.find_all("a")
    hrefs = [t.get("href") for t in a_list if t.get("class") and t.get("class")[0]=="j_th_tit"]
    title_list.extend(hrefs)
#print(title_list)

[u'/p/4044242904', u'/p/4002886965', u'/p/4050390715', u'/p/4050462515', u'/p/4050388506', u'/p/4050410601', u'/p/4047071182', u'/p/4050235265', u'/p/3851625451', u'/p/3965913323', u'/p/4050069145', u'/p/4050004708', u'/p/2934603539', u'/p/4048025883', u'/p/4047343020', u'/p/4048827233', u'/p/2975146875', u'/p/4049609957', u'/p/4049705960', u'/p/4005719039', u'/p/4049277272', u'/p/3988123090', u'/p/4049714732', u'/p/4049673878', u'/p/4034745290', u'/p/3905628371', u'/p/3529970849', u'/p/4049617130', u'/p/4042141617', u'/p/4032430137', u'/p/3990011695', u'/p/4028432545', u'/p/4049475057', u'/p/3779402788', u'/p/3464733161', u'/p/3765086316', u'/p/4041878563', u'/p/3790062315', u'/p/4041905744', u'/p/3851154684', u'/p/3950107421', u'/p/4010988787', u'/p/3999351710', u'/p/4048684778', u'/p/4024788621', u'/p/3246566913', u'/p/4048723085', u'/p/4039897877', u'/p/4043501331', u'/p/3723810101', u'/p/4047865597', u'/p/3777230698', u'/p/4044498330', u'/p/4042362169', u'/p/4047557479', u'/p/4046

In [72]:
#进行url拼装
import urlparse
#url拼接的例子
print urlparse.urljoin('http://somehost.com/', '../other/path')
#进行拼接
title_list = [urlparse.urljoin( main_url , t) for t in title_list]
#print title_list

http://somehost.com/../other/path
[u'http://tieba.baidu.com/p/4044242904', u'http://tieba.baidu.com/p/4002886965', u'http://tieba.baidu.com/p/4050390715', u'http://tieba.baidu.com/p/4050462515', u'http://tieba.baidu.com/p/4050388506', u'http://tieba.baidu.com/p/4050410601', u'http://tieba.baidu.com/p/4047071182', u'http://tieba.baidu.com/p/4050235265', u'http://tieba.baidu.com/p/3851625451', u'http://tieba.baidu.com/p/3965913323', u'http://tieba.baidu.com/p/4050069145', u'http://tieba.baidu.com/p/4050004708', u'http://tieba.baidu.com/p/2934603539', u'http://tieba.baidu.com/p/4048025883', u'http://tieba.baidu.com/p/4047343020', u'http://tieba.baidu.com/p/4048827233', u'http://tieba.baidu.com/p/2975146875', u'http://tieba.baidu.com/p/4049609957', u'http://tieba.baidu.com/p/4049705960', u'http://tieba.baidu.com/p/4005719039', u'http://tieba.baidu.com/p/4049277272', u'http://tieba.baidu.com/p/3988123090', u'http://tieba.baidu.com/p/4049714732', u'http://tieba.baidu.com/p/4049673878', u'htt

In [80]:
title_list = title_list[0:5]#节省时间 ，暂时先用前几个帖子
image_url_list = [] #保存所有获取到图片的url 供之后下载
for t_url in title_list: #处理每一个主题页面
    pn_num = 1 #先假设只有一页
    i=0
    while i< pn_num :
        i = i+1
        #设置页面参数
        param = {"see_lz" : 1 ,"pn":i+1}  #只抓取楼主的内容
        req = requests.get(t_url,param)
        page = req.text
        soup = BeautifulSoup(page,'html.parser')
        im_list = soup.find_all("img") #先获取图片元素
        im_list = [t.get("src") for t in im_list if t.get("class") and t.get("class")[0]=="BDE_Image"]
        image_url_list.extend(im_list)
        
        #获取真实的页数
        pnc = soup.find_all("span")
        pnc = [p.text for p in pnc if p.get("class")and p.get("class")[0] == "red"]
        pn_num =  int(pnc[1]) 
        
print(image_url_list)

[u'http://imgsrc.baidu.com/forum/w%3D580/sign=7eb69da145166d223877159c76220945/3261a38b87d6277fb56228522e381f30e824fccc.jpg', u'http://imgsrc.baidu.com/forum/w%3D580/sign=65747f05982f07085f052a08d925b865/4e8da964034f78f01594f7b97f310a55b2191cb9.jpg']


In [82]:
#开始下载
for img_url in image_url_list :
    saveImage(img_url , getT()+".jpg")

保存文件7663674.jpg

保存文件7663702.jpg

